# Two showers sideband run by run

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#pi0scaling = {"val": 0, "suffix": "", "title": "No pi0 Scaling"} #no scaling
#pi0scaling = {"val": 1, "suffix": "_pi0flat", "title": "0.759 pi0 Scaling"} #flat scaling
pi0scaling = {"val": 2, "suffix": "_pi0e035", "title": "Energy-dependent pi0 Scaling"} #energy-dependent


In [ ]:
from matplotlib import pyplot as plt

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
plt.rcParams.update(params)

from load_data_run123 import *
from unblinding_far_sideband import *

In [ ]:
import sys
import localSettings as ls
print(ls.main_path)
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
samples = load_data_run123(which_sideband='2plus_showers', return_plotter=False, pi0scaling=pi0scaling["val"])

In [ ]:
plotter_runbyrun = {}
for run in [1, 2, 3, 123, 12]:
    this_weights, this_pot = get_weights(run)
    plotter_runbyrun[run] = plotter.Plotter(samples, this_weights, pot=this_pot)

In [ ]:
TRAINVAR = ["shr_score","tksh_distance","tksh_angle",
            "shr_tkfit_dedx_max",
            "trkfit","trkpid",
            "subcluster","shrmoliereavg",
            "trkshrhitdist2","hits_ratio",
            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
            "CosmicIPAll3D","CosmicDirAll3D"]

pvaluetypes = ['pvalue','pvaluediag','pvaluestatonly']

# Study Np selection on the 2+ shower sideband, after unblinding

In [ ]:
from unblinding_far_sideband import *
for run in [1, 2, 3, 12, 123]:
#for run in [3,12,123]:
    for stage in [1,3,4,5,6,7,8,9]:
    #for stage in [0]:
        print('\nRun %i, Stage %i'%(run, stage))
        this_folder = ls.plots_path+'PELEE_two_showers_sideband_np_selection_runbyrun/'
        this_folder += 'run_{}/'.format(run)
        this_folder += 'stage_{}'.format(stage)
        this_folder += (pi0scaling["suffix"]+'/')
        !mkdir -p $this_folder

        this_query = stages_queries_two_plus_showers[stage]
        if run in [1, 2, 3, 12]:
            this_query += f' and run{run}==True'
        this_title = f'Two+ showers sideband - run {run} - ' + pi0scaling["title"] + '\n' +\
                       stages_titles_two_plus_showers[stage]

        pvalues = [[] for _ in range(len(pvaluetypes))]

        for plot_variable in plot_variables:
            VARIABLE, BINS, RANGE, XTIT = plot_variable[0:4]
            #if VARIABLE != 'n_showers_contained':
            #if VARIABLE != 'n_tracks_contained':
            #if len(plot_variable)>4 or VARIABLE not in TRAINVAR:
            #if XTIT != 'normalization':
            #    continue
            print(VARIABLE, BINS, RANGE, XTIT)
            fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
                VARIABLE,   
                query=this_query,
                kind="event_category",
                draw_sys=True,
                stacksort=3,
                title=XTIT,
                bins=BINS,
                range=RANGE,
            )[0:3]

            if len(plot_variable) == 6:
                if plot_variable[5] is True:
                    ax1.set_yscale('log')
            else:
                ax1.set_ylim(0, ax1.get_ylim()[1]*1.8)

            ax1.set_title(this_title, loc='left')

            plt.tight_layout()
            save_path = this_folder+VARIABLE
            if len(plot_variable) >= 5:
                save_path += ('_' + plot_variable[4])
            fig.savefig(save_path + '.png', dpi=250)    
            plt.close()
            
            # store pvalues
            if len(plot_variable)==4 and VARIABLE in TRAINVAR:
                for idx, pvt in enumerate(pvaluetypes):
                    pvalues[idx].append(plotter_runbyrun[run].stats[pvt])
        # make pvalue plots
        #print(pvalues)
        for idx, pvt in enumerate(pvaluetypes):
            if len(pvalues[idx])!=16: continue
            plt.figure(figsize=(9, 7))
            plt.hist(pvalues[idx], bins=10, range=(0,1))
            plt.title(this_title, loc='left')
            plt.xlabel(pvt)
            plt.ylabel("number of BDT input variables")
            plt.savefig(this_folder + pvt + '.png', dpi=250)
            plt.close()

In [ ]:
#samples['data'].query(BDTCQ)[["run","sub","evt","reco_e","pi0_score","nonpi0_score","n_showers_contained","n_tracks_contained"]]
samples['data'].query(BDTCQ)[["run","sub","evt","reco_e","pi0_score","nonpi0_score","n_showers_contained","n_tracks_contained","reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z"]]

In [ ]:
from unblinding_far_sideband import *
for run in [3]:
    for stage in [4, 5]:            
        this_folder = ls.plots_path+'PELEE_two_showers_sideband_np_selection_runbyrun/'
        this_folder += 'run_{}/'.format(run)
        this_folder += 'stage_{}/'.format(stage)
        !mkdir -p $this_folder

        this_query = stages_queries_two_plus_showers[stage]
        if run in [1, 2, 3]:
            this_query += f' and run{run}==True'
        this_title = f'Two+ showers sideband - run {run}\n' +\
                        r'$\nu_e$ preselection and N-showers contained >= 2' +\
                        '\n' + stages_titles_two_plus_showers[stage]

        for plot_variable in plot_variables:
            VARIABLE, BINS, RANGE, XTIT = plot_variable[0:4]
            print(VARIABLE, BINS, RANGE, XTIT)
            fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
                VARIABLE,   
                query=this_query,
                kind="event_category",
                draw_sys=True,
                stacksort=3,
                title=XTIT,
                bins=BINS,
                range=RANGE,
            )[0:3]

            if len(plot_variable) == 6:
                if plot_variable[5] is True:
                    ax1.set_yscale('log')
            else:
                ax1.set_ylim(0, ax1.get_ylim()[1]*1.6)

            ax1.set_title(this_title, loc='left')

            plt.tight_layout()
            save_path = this_folder+VARIABLE
            if len(plot_variable) >= 5:
                save_path += ('_' + plot_variable[4])
            fig.savefig(save_path + '.png', dpi=250)    
            plt.close()

# Study Np selection low-medium energy on the 2+ shower sideband, after unblinding

In [ ]:
from unblinding_far_sideband import *
for run in [1, 2, 3, 123]:
    for stage in [0]:        
        this_folder = ls.plots_path+'PELEE_two_showers_sideband_np_selection_low_medium_energy_runbyrun/'
        this_folder += 'run_{}/'.format(run)
        this_folder += 'stage_{}/'.format(stage)
        !mkdir -p $this_folder

        this_query = stages_queries_two_plus_showers_low_medium_energy[stage]
        if run in [1, 2, 3]:
            this_query += f' and run{run}==True'
        this_title = f'Two+ showers sideband - run {run}\n' +\
                        r'$\nu_e$ preselection and N-showers contained >= 2' +\
                        '\n' + stages_titles_two_plus_showers_low_medium_energy[stage]

        for plot_variable in plot_variables:
            VARIABLE, BINS, RANGE, XTIT = plot_variable[0:4]
            print(VARIABLE, BINS, RANGE, XTIT)
            fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
                VARIABLE,   
                query=this_query,
                kind="event_category",
                draw_sys=True,
                stacksort=3,
                title=XTIT,
                bins=BINS,
                range=RANGE,
            )[0:3]

            if len(plot_variable) == 6:
                if plot_variable[5] is True:
                    ax1.set_yscale('log')
            else:
                ax1.set_ylim(0, ax1.get_ylim()[1]*1.6)

            ax1.set_title(this_title, loc='left')

            plt.tight_layout()
            save_path = this_folder+VARIABLE
            if len(plot_variable) >= 5:
                save_path += ('_' + plot_variable[4])
            fig.savefig(save_path + '.png', dpi=250)    
            plt.close()